In [103]:
!pip3 install numpy
!pip3 install scipy
!pip3 install pandas
!pip3 install multiprocessing
!pip3 install statsmodels
!pip3 install matplotlib

import random 
import numpy as np
import scipy as sp
import pandas as pd
from multiprocessing import Pool
from statsmodels.stats.multitest import multipletests as mult  #MULTIPLE TESTING MODULE 
from matplotlib import pyplot as plt


^C
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached multiprocessing-2.6.2.1.tar.gz (108 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [7 lines

In [ ]:
MaxReps=100000
PREFIX="REACTOME_"#DB to explore for enrichment (see the DB file for other formats "GO_", others...)

In [ ]:
#'''
file="/Users/dolanpt/laptop/GitHub/doggo/msigdb.v7.1.symbols.gmt"#CHANGE THIS TO YOUR LOCATION. RUN BLOCK ONCE (to make pickles) THEN COMMENT OUT. 
with open(file,'r') as inf:
    alllines=[line.strip().split("\t") for line in inf]
    DF=pd.concat([pd.DataFrame(zip([l[0]]*len(l[2:]),l[2:]),columns=["term","genes"]) for l in alllines])
pd.to_pickle(DF,"/Users/dolanpt/laptop/GitHub/doggo/msigdb.pkl")#'''

In [ ]:
MSigDB=pd.read_pickle("/Users/dolanpt/laptop/GitHub/doggo/msigdb.pkl")#CHANGE TO YOUR LOCALE (MATCH ABOVE)

In [ ]:
#This inputData file should be a long table with genes as rows and the data (parameters for filtering and scoring (pval etc.)) for each as columns. 

inputData= "~/Downloads/WT_RD_37.csv"
data=pd.read_table(inputData,delimiter=",")
data

,Unnamed: 0,ens_gene,ext_gene,target_id,pval,qval,b,se_b,mean_obs,var_obs,tech_var,sigma_sq,smooth_sigma_sq,final_sigma_sq
0,1,ENSG00000161016,RPL8,ENST00000394920.6,5.285298e-27,3.730839e-22,1.180117,0.109671,8.346138,0.472044,0.009933,0.001795,0.002095,0.002095
1,2,ENSG00000142541,RPL13A,ENST00000467825.2,5.820181e-15,1.369469e-10,-0.556303,0.071250,8.513118,0.106395,0.002855,0.002001,0.002221,0.002221
2,3,ENSG00000184557,SOCS3,ENST00000330871.3,5.183619e-15,1.369469e-10,0.519025,0.066351,6.953304,0.092666,0.001382,0.002924,0.003021,0.003021
3,4,ENSG00000138379,MSTN,ENST00000260950.5,2.785983e-13,4.916494e-09,-0.367197,0.050271,7.816400,0.046068,0.000508,0.001177,0.002019,0.002019
4,5,ENSG00000144713,RPL32,ENST00000457131.1,4.414963e-13,6.232956e-09,-0.827729,0.114293,8.127492,0.232328,0.011056,-0.005131,0.002007,0.002007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70584,70585,NaN,NaN,ENST00000637549.1,8.482912e-01,9.999720e-01,-0.134979,0.705589,4.139645,0.181924,0.377386,-0.113610,0.120469,0.120469
70585,70586,NaN,NaN,ENST00000637733.1,5.978526e-01,9.999720e-01,-0.091403,0.173279,5.555590,0.017612,0.018043,0.004198,0.011983,0.011983
70586,70587,NaN,NaN,ENST00000644141.2,9.618411e-01,9.999720e-01,0.027082,0.566061,5.089839,0.075538,0.298643,-0.185704,0.021782,0.021782
70587,70588,NaN,NaN,ENST00000645777.1,7.704769e-01,9.999720e-01,0.263748,0.904016,2.990525,0.111174,0.044182,0.087797,0.773063,0.773063


In [181]:
def computeEnrichment(t,d,P="b",ids="ext_gene"):#
    print(t)
    DBgenes=MSigDB.loc[MSigDB.term==t].genes
    #print(DBgenes)
    totalMean=d[P].values.mean()
    print(totalMean)
    targets=d.loc[d[ids].isin(DBgenes.values)]
    #print(targets)
    nrowS=targets.shape[0]
    if nrowS>2:#If more that two genes in term set...
        #print(t)
        meanSelect=totalMean-targets[P].values.mean()
        print(meanSelect)
        nrowT=d.shape[0]
        positives=0
        denom=0
        if meanSelect < 0:#choose which tail to test...
            while (positives<30)&(denom<MaxReps):
                denom+=1
                diffSample=totalMean-d[P].iloc[np.random.randint(0,nrowT,nrowS)].mean()
                positives+=int(diffSample<meanSelect)
        else:
            while (positives<30)&(denom<MaxReps):
                denom+=1                
                diffSample=totalMean-d[P].iloc[np.random.randint(0,nrowT,nrowS)].mean()
                positives+=int(diffSample>meanSelect)
        #print(positives/denom)
        #print((meanSelect-totalMean)/totalMean)
    else:
        print("No hits. Skipping Term.")
        return(1.0,1.0,1.0)
    return([positives/denom, meanSelect, nrowS])

In [189]:
#This is the main block, and needs to be edited based on your data and the parameter (score column you will be sampling in your data.)
def f(term):
    DF=pd.DataFrame([term,computeEnrichment(term,data,"b")],index=["term","pBS"]).transpose()
    return(DF)

#Run across all terms. 
StatResults=pd.DataFrame()
for term in MSigDB.term.unique():
    if term.startswith(PREFIX):
        result=f(term)
        StatResults=pd.concat([StatResults,pd.DataFrame([term,result.pBS[0][0],result.pBS[0][1],result.pBS[0][2]]).transpose()], ignore_index=True)
StatResults.columns=["term","pBS","mean","N"]            

StatResults['adjPbs']=mult(StatResults.pBS)[1]
StatResults
StatResults.to_csv("WT_RD_37_pBS.csv")

REACTOME_GLYCOGEN_BREAKDOWN_GLYCOGENOLYSIS
-0.1917668403446273
0.05225660712352531
REACTOME_TRANSLATION
-0.1917668403446273
-0.006217397423342902
REACTOME_PYRIMIDINE_CATABOLISM
-0.1917668403446273
0.12670968755635773
REACTOME_INHIBITION_OF_THE_PROTEOLYTIC_ACTIVITY_OF_APC_C_REQUIRED_FOR_THE_ONSET_OF_ANAPHASE_BY_MITOTIC_SPINDLE_CHECKPOINT_COMPONENTS
-0.1917668403446273
0.11122012942498413
REACTOME_PYRUVATE_METABOLISM_AND_CITRIC_ACID_TCA_CYCLE
-0.1917668403446273
0.10607381653432266
REACTOME_APOPTOTIC_CLEAVAGE_OF_CELLULAR_PROTEINS
-0.1917668403446273
0.011154114454577785
REACTOME_ACTIVATION_OF_THE_PRE_REPLICATIVE_COMPLEX
-0.1917668403446273
0.16782544760552506
REACTOME_PROCESSING_OF_INTRONLESS_PRE_MRNAS
-0.1917668403446273
-0.16555077809679028
REACTOME_GAP_JUNCTION_DEGRADATION
-0.1917668403446273
-0.14395605386615012
REACTOME_BASE_EXCISION_REPAIR
-0.1917668403446273
0.11917031942508402
REACTOME_BILE_ACID_AND_BILE_SALT_METABOLISM
-0.1917668403446273
0.1359723000760368
REACTOME_SYNTHESIS_OF

KeyboardInterrupt: 